# Library Import

In [14]:
import pandas as pd 
import yfinance as yf
import polygon 
import numpy as np
from dotenv import load_dotenv
import os
import requests

## Yahoo Finance

In [15]:
# History Fetching
Ticker = yf.Ticker('VSA')
Ticker.history(period='max')

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [6]:
# Download Data
yf.download(['ADS1.DU'], period='max')

[*********************100%***********************]  1 of 1 completed

YF.download() has changed argument auto_adjust default to True


Price,Close,High,Low,Open,Volume
Ticker,ADS1.DU,ADS1.DU,ADS1.DU,ADS1.DU,ADS1.DU
Date,,,,,
2020-02-06,136.309311,136.309311,136.309311,136.309311,0
2020-02-07,135.349396,135.349396,135.349396,135.349396,0
2020-02-10,135.349396,135.349396,135.349396,135.349396,0
2020-02-11,135.349396,135.349396,135.349396,135.349396,0
2020-02-12,135.349396,135.349396,135.349396,135.349396,0
...,...,...,...,...,...
2025-03-24,108.000000,110.000000,108.000000,110.000000,0
2025-03-25,110.000000,111.000000,109.000000,109.000000,0


In [13]:
import asyncio
import time
from concurrent.futures import ThreadPoolExecutor

def test_concurrent_requests():
    
    max_workers = 50  # Test with different numbers
    symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'JPM', 'V', 'WMT', 
            'BAC', 'JNJ', 'PG', 'MA', 'HD', 'DIS', 'PFE', 'KO', 'NFLX', 'CSCO', 
            'ABBV', 'PEP', 'T', 'MRK', 'INTC', 'CVX', 'MCD', 'WFC', 'ABT', 'VZ', 
            'TMO', 'CRM', 'BMY', 'UNH', 'LOW', 'NKE', 'HON', 'ORCL', 'ACN', 'DHR', 
            'LIN', 'PM', 'UPS', 'RTX', 'AMD', 'IBM', 'QCOM', 'TXN', 'INTU', 'ADP']
    
    def fetch_data(symbol):
        try:
            start_time = time.time()
            ticker = yf.Ticker(symbol)
            data = ticker.history(period='max')
            end_time = time.time()
            return {
                'symbol': symbol,
                'success': True,
                'time_taken': end_time - start_time,
                'data_points': len(data)
            }
        except Exception as e:
            return {
                'symbol': symbol,
                'success': False,
                'error': str(e)
            }
    
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(fetch_data, symbol) for symbol in symbols]
        for future in futures:
            results.append(future.result())
    
    # Print results
    print(f"\nTesting with {max_workers} concurrent workers:")
    print("=" * 50)
    for result in results:
        if result['success']:
            print(f"{result['symbol']}: Success - {result['time_taken']:.2f}s - {result['data_points']} data points")
        else:
            print(f"{result['symbol']}: Failed - {result['error']}")

# Run the test
test_concurrent_requests()



Testing with 50 concurrent workers:
AAPL: Success - 0.63s - 11183 data points
MSFT: Success - 0.38s - 9857 data points
GOOGL: Success - 0.24s - 5205 data points
AMZN: Success - 0.28s - 7031 data points
META: Success - 0.24s - 3253 data points
TSLA: Success - 0.21s - 3730 data points
NVDA: Success - 0.76s - 6606 data points
JPM: Success - 0.73s - 11371 data points
V: Success - 0.29s - 4304 data points
WMT: Success - 1.32s - 13277 data points
BAC: Success - 1.66s - 13157 data points
JNJ: Success - 2.03s - 15936 data points
PG: Success - 2.22s - 15936 data points
MA: Success - 0.45s - 4760 data points
HD: Success - 1.62s - 10988 data points
DIS: Success - 2.32s - 15936 data points
PFE: Success - 2.09s - 13337 data points
KO: Success - 2.23s - 15936 data points
NFLX: Success - 0.50s - 5769 data points
CSCO: Success - 0.79s - 8862 data points
ABBV: Success - 0.31s - 3098 data points
PEP: Success - 1.91s - 13337 data points
T: Success - 1.76s - 10439 data points
MRK: Success - 2.29s - 15936

## Polygon API

In [7]:
# Load environment variables
load_dotenv()

# Fetch OHLCV data
# Get Polygon API key
api_key = os.environ.get('POLYGON_API_KEY')
if not api_key:
    raise ValueError("POLYGON_API_KEY environment variable not found")

# API parameters
params = {
    'ticker': 'AGM.A',
    'multiplier': 1,
    'timespan': 'day',
    'adjusted': False,
    'sort': 'asc',
    'apiKey': api_key,
    'from': '2023-01-01',  # Add date range
    'to': '2023-12-31'
}

# Make API request
url = "https://api.polygon.io/v2/aggs/grouped/locale/us/market/stocks/2023-01-01"
response = requests.get(url, params=params)

# Check response status
if response.status_code != 200:
    raise Exception(f"API request failed with status code {response.status_code}")

try:
    response_json = response.json()
except requests.exceptions.JSONDecodeError as e:
    raise Exception(f"Failed to decode JSON response: {str(e)}")

In [14]:
from polygon import RESTClient
# Load environment variables
load_dotenv()

# Fetch OHLCV data
# Get Polygon API key
api_key = os.environ.get('POLYGON_API_KEY')
if not api_key:
    raise ValueError("POLYGON_API_KEY environment variable not found")

# API parameters
params = {
    'ticker': 'AGM.A',
    'apiKey': api_key,
}

# Make API request
url = "https://api.polygon.io/v3/reference/tickers/"
response = requests.get(url, params=params)

# Check response status
if response.status_code != 200:
    raise Exception(f"API request failed with status code {response.status_code}")

try:
    response_json = response.json()
except requests.exceptions.JSONDecodeError as e:
    raise Exception(f"Failed to decode JSON response: {str(e)}")


In [17]:
response_json['results'][0]

{'ticker': 'AGM.A',
 'name': 'Federal Agricultural Mortgage Corporation Class A Voting',
 'market': 'stocks',
 'locale': 'us',
 'primary_exchange': 'XNYS',
 'type': 'CS',
 'active': True,
 'currency_name': 'usd',
 'cik': '0000845877',
 'composite_figi': 'BBG000G8Y1Z6',
 'share_class_figi': 'BBG001SCWN95',
 'last_updated_utc': '2025-03-28T00:00:00Z'}

In [28]:
import pandas_market_calendars as mcal
import pytz
from datetime import datetime
# Get NYSE calendar
nyse = mcal.get_calendar('NYSE')

# Get current NY time
ny_tz = pytz.timezone('America/New_York')
today = datetime.strptime('2025-05-02', '%Y-%m-%d')
end_date = datetime.strptime('2025-05-05', '%Y-%m-%d')
# Get market schedule
schedule = nyse.schedule(start_date=today, end_date=end_date)

print(len(schedule))

# Check if current time is past market close
current_time = datetime.now(pytz.timezone('America/New_York'))
last_market_close = schedule.iloc[-1]['market_close'].tz_convert('America/New_York') 

# Get market hours
market_open = schedule.iloc[0]['market_open'].tz_convert('America/New_York')
market_close = schedule.iloc[0]['market_close'].tz_convert('America/New_York')
    
trading_hours = pd.date_range(
    start=market_open,
    end=market_close,
    freq='1d',
    tz='America/New_York'
)[1:].tolist()



2
